# Modelo para clasificar el BOE

Una vez que hayamos clasificado los 2000 BOEs que se encuentran en el fichero "[BOEs juntos](https://drive.google.com/open?id=1zz6F-fr1d-FHAxeZWWN2ZNgtexC139eQ-nFRV3BXLUk)".

Este desarrollo está basado parcialmente en los siguientes papers:
1. https://medium.com/data-from-the-trenches/text-classification-the-first-step-toward-nlp-mastery-f5f95d525d73
2. https://machinelearningmastery.com/load-machine-learning-data-python/





#1. Cargamos las bibliotecas que son necesarias

In [0]:
# Nos ayudará a vectorizar los textos
from sklearn.feature_extraction.text import CountVectorizer

# Biblioteca de Expresiones Regulares. Nos ayudará a limpiar el texto
import re

# Para importar y poder trabajar con los CSV
import csv

import numpy as np
import pandas as pd

!pip install -U nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


In [0]:
from sklearn.metrics import accuracy_score

# Para esta iteracción vamos a usar el clasificador del Support Vector Machine
# Otras opciones podrían ser
# * Logistic Regressions
# * Naive Bayes
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

#2. Creamos la función que nos ayudará a preparar los textos:
* Quita espacios repetidos
* Quitar caracteres extraños puntos, comas paréntesis, etc
* Quita enters, tabuladores
* Coloca todo en minúsculas

In [0]:
def clean_text(serie):
    """
    Adapta el texto dado.

    Pasos:
    - Quita signos de puntuación
    - Pone todo en minúsculas
    """
    
    text = serie[0]
        
    # Quita los [\], ['] y ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)    
    
    # Pone todo en minúsculas
    text = text.strip().lower()
    
    # Reemplaza todos los caracteres "raros" por espacios
    filters='«»¡!"\'#$€%&()*+,-./:;<=>¿?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    serie[0] = text

    return serie

# 3. Limpiamos el data set de entrenamiento

In [0]:
# Cargamos el CSV

# Este la dirección en donde esta el fichero CSV
filename = './BOES-juntos.csv'

train = pd.read_csv(filename)

# Hay veces que el excel tiene líneas en blanco
# y estas son interpretadas como non/available N/A
# el método dropna() del DataFrame de pandas los quita
train = train.dropna()

# Aplicamos la función de limpieza
train = train.apply(clean_text, axis=1)

#4. Limpiamos el data set a procesar

In [0]:
import os
import time

# Este la dirección en donde esta el fichero CSV
compressed_filename = './t.csv.xz';

# Al final, el fichero obtenido es u.csv
filename = compressed_filename.replace(".xz", "")
print(filename)

# Si existe el fichero u.csv existe no lo descomprimimos
if os.path.isfile(filename) == False:
  # Descomprimir el CSV
  !unxz $compressed_filename

# Cargamos el CSV
ts = time.time()
process = pd.read_csv(filename, error_bad_lines=False, header=None)

print ("Hemos tardado " + str(time.time() - ts) + " segundos en cargar el CSV")
ts = time.time()

# Hay veces que el excel tiene líneas en blanco
# y estas son interpretadas como non/available N/A
# el método dropna() del DataFrame de pandas los quita
process = process.dropna()

# Aplicamos la función de limpieza
process = process.apply(clean_text, axis=1)

print ("Hemos tardado " + str(time.time() - ts) + " segundos en procesar CSV")


# 5. Cargamos el vectorizador
### Junto con el listado de stopwords en español

In [0]:
import pickle

vectorizer = pickle.load(open("BOE_fit_words.pickle", "rb"))



#5.1. Vectorizamos el conjunto de entrenamiento

In [0]:
# Vectorizamos el conjunto de datos de entrenamiento (train)

training_features = vectorizer.transform(train["Texto"])


#5.2 Vectorizamos el conjunto de proceso

In [0]:
# Vectorizamos el conjunto de datos de proceso (proces)
ts = time.time()
process_features = vectorizer.transform(process[2])
print ("Hemos tardado " + str(time.time() - ts) + " segundos en vectorizar")

Hemos tardado 465.43222522735596 segundos en vectorizar


# 6. Creamos el modelo y... lo ejecutamos

In [0]:
#@title Texto de título predeterminado
# Creamos el modelo. Linear Support Vector Machine Classifier
model = LinearSVC(max_iter=5000)
model = MultinomialNB()

# Entrenamos
model.fit(training_features, train["Categoria"])

# Hacemos una predicción
y_pred = model.predict(process_features)

#acc = accuracy_score(test["Categoria"], y_pred)

#print("Accuracy on the BOE dataset: {:.2f}".format(acc*100))

y_pred

array([11, 11, 11, ..., 11, 20, 19])

In [0]:
import gc


In [0]:
gc.collect()
i = np.array([[1, "acuerdo"],
              [2, "anuncios"],
              [3, "aprobacion"],
              [4, "competencias"],
              [5, "concurso"],
              [6, "constitucional"],
              [7, "convenio"],
              [8, "correccion"],
              [9, "divisas"],
              [10, "educacion"],
              [11, "empleo/oposicion"],
              [12, "extravio"],
              [13, "ley"],
              [14, "nombramientos"],
              [15, "politica exterior"],
              [16, "politica interior"],
              [17, "premios"],
              [18, "resolucion"],
              [19, "subasta"],
              [20, "subvencion"]])
i

array = np.hstack((np.vstack((process[1].to_numpy())), np.vstack((y_pred))))


In [0]:
gc.collect()

np.save("./result", array)

In [0]:
!ls -l